In [1]:
import pandas as pd

In [2]:
hnposts = pd.read_csv("../data/raw/HN_posts_year_to_Sep_26_2016.csv")

In [3]:
hnposts.head()

,id,title,url,num_points,num_comments,author,created_at
0,12579008,You have two days to comment if you want stem ...,http://www.regulations.gov/document?D=FDA-2015...,1,0,altstar,9/26/2016 3:26
1,12579005,SQLAR the SQLite Archiver,https://www.sqlite.org/sqlar/doc/trunk/README.md,1,0,blacksqr,9/26/2016 3:24
2,12578997,What if we just printed a flatscreen televisio...,https://medium.com/vanmoof/our-secrets-out-f21...,1,0,pavel_lishin,9/26/2016 3:19
3,12578989,algorithmic music,http://cacm.acm.org/magazines/2011/7/109891-al...,1,0,poindontcare,9/26/2016 3:16
4,12578979,How the Data Vault Enables the Next-Gen Data W...,https://www.talend.com/blog/2016/05/12/talend-...,1,0,markgainor1,9/26/2016 3:14


In [4]:
hnposts.shape

(293119, 7)

In [5]:
hnposts["title"][0]

'You have two days to comment if you want stem cells to be classified as your own'

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer(max_df=0.9, min_df=2, stop_words="english")

In [8]:
dtm = cv.fit_transform(hnposts["title"])

In [9]:
dtm

<293119x41917 sparse matrix of type '<class 'numpy.int64'>'
	with 1534334 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
LDA = LatentDirichletAllocation(n_components=8, random_state=42)

In [ ]:
LDA.fit(dtm)

In [ ]:
len(cv.get_feature_names())
import random

random_word_id = random.randint(0, 41917)
cv.get_feature_names()[random_word_id]

In [ ]:
len(LDA.components_)
LDA.components_.shape

In [ ]:
single_topic = LDA.components_[0]

In [ ]:
single_topic.argsort()

In [ ]:
top_fifteen_words = single_topic.argsort()[-15:]
top_fifteen_words

In [ ]:
for index in top_fifteen_words:
    print(cv.get_feature_names()[index])

In [ ]:
for index, topic in enumerate(LDA.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print("\n\n\n")

In [ ]:
topic_results = LDA.transform(dtm)

In [ ]:
topic_results.shape

In [ ]:
topic_results[0].round(3)

In [ ]:
topic_results.argmax(axis=1) + 1

In [ ]:
hnposts["Topic"] = topic_results.argmax(axis=1)

In [ ]:
hnposts.head(50)

In [ ]:
data = cv.fit_transform(hnposts["title"])
data


In [ ]:
result = LDA.transform(cv.transform(["Six Recipes for Software Managers"]))
result.argmax()

In [ ]:
def predictCluster(title):
    result = LDA.transform(cv.transform([title]))
    return result.argmax()

In [ ]:
predictCluster("Kickstarting a new wave of self driving car companies")